# --- Day : 11 Seating System ---

https://adventofcode.com/2020/day/11

I would like to come back and revisit this one.   
The solution I've come up with is WAY too slow...

In [1]:
path = '../inputs/'

## Part 1

### Map Legend
floor = .   
empty seat = L   
occupied seat = #   

In [2]:
import copy
import numpy as np

In [3]:
def get_initial_state(filename):
    
    with open(path + filename) as file:
        # Set the floor as '9'
        state = [row.strip().replace('L', '0').replace('.', '9') for row in file]
        # Convert to numpy array, to be able to test for equality between states
        state = np.array([[int(char) for char in row] for row in state])
        # Convert 9s to nans
        state = state.astype('float')
        state[state==9] = np.nan
    
    return state

In [4]:
initial_state = get_initial_state('example_seat_layout.txt')
initial_state

array([[ 0., nan,  0.,  0., nan,  0.,  0., nan,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., nan,  0.,  0.],
       [ 0., nan,  0., nan,  0., nan, nan,  0., nan, nan],
       [ 0.,  0.,  0.,  0., nan,  0.,  0., nan,  0.,  0.],
       [ 0., nan,  0.,  0., nan,  0.,  0., nan,  0.,  0.],
       [ 0., nan,  0.,  0.,  0.,  0.,  0., nan,  0.,  0.],
       [nan, nan,  0., nan,  0., nan, nan, nan, nan, nan],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., nan,  0.,  0.,  0.,  0.,  0.,  0., nan,  0.],
       [ 0., nan,  0.,  0.,  0.,  0.,  0., nan,  0.,  0.]])

In [5]:
def num_adjacents(array, i, j):
    
    num_rows = len(array)
    num_cols = len(array[0])
    adjacents = []
    
    for row in range(max(0, i-1), min(i+1+1, num_rows)):
        for col in range(max(0, j-1), min(j+1+1, num_cols)):
            if not (row == i and col == j):
                adjacents.append(array[row][col])
    
    return np.nansum(adjacents)

In [6]:
def change_state(state, new_state, func=num_adjacents, tol=4):
    
    for i in range(len(state)):
        for j in range(len(state[0])):

            if not np.isnan(state[i][j]):
                if func(state, i, j) == 0:
                    new_state[i][j] = 1
                elif func(state, i, j) >= tol:
                    new_state[i][j] = 0
                else:
                    new_state[i][j] == state[i][j]
    
    return new_state

In [7]:
def run_sims(state, num_calls=1):

    new_state = copy.deepcopy(state)     
    new_state = change_state(state, new_state, func=num_adjacents, tol=4)
    
    if np.array_equal(new_state, state, equal_nan=True):
        return np.nansum(new_state), num_calls
    
    else:
#         print('round ', calls)
#         print(np.nansum(new_state))
#         print('state\n', state, '\nnew_state\n', new_state,'\n\n')
    
        return run_sims(new_state, num_calls+1)


In [8]:
run_sims(initial_state) # Should return 37

(37.0, 6)

In [9]:
run_sims(get_initial_state('seat_layout.txt'))

(2108.0, 71)

## Part 2

In [10]:
def count_in_view(array, i, j):
    """Return number of occupied seats in view."""
    num_in_view = 0
    
    # Top
    if i > 0:
        for r in range(i-1, -1, -1):
            if array[r, j] == 1:
                num_in_view += 1
                break
            if array[r, j] == 0:
                break

    # Top right diagnonal
    if i > 0 and j < len(array[0]):
        rows = list(range(i-1, -1, -1))
        cols = list(range(j+1, len(array[0])))        
        for r, c in zip(rows, cols):
            if array[r, c] == 1:
                num_in_view += 1
                break
            if array[r, c] == 0:
                break
    
    # Right
    if j < len(array[0]):
        for c in range(j+1, len(array[0])):
            if array[i, c] == 1:
                num_in_view += 1
                break
            if array[i, c] == 0:
                break
    
    # Bottom right diagonal
    if i < len(array) - 1 and j < len(array[0]):
        rows = list(range(i+1, len(array)))
        cols = list(range(j+1, len(array[0]-1)))
        for r, c in zip(rows, cols):
            if array[r, c] == 1:
                num_in_view += 1
                break
            if array[r, c] == 0:
                break
    
    # Bottom
    if i < len(array):
        for r in range(i+1, len(array)):
            if array[r, j] == 1:
                num_in_view += 1
                break
            if array[r, j] == 0:
                break

    # Bottom left diagonal
    if i < len(array) - 1 and j > 0:
        rows = list(range(i+1, len(array)))
        cols = list(range(j-1, -1, -1))
        for r, c in zip(rows, cols):
            if array[r, c] == 1:
                num_in_view += 1
                break
            if array[r, c] == 0:
                break
    
    # Left
    if j > 0:
        for c in range(j-1, -1, -1):
            if array[i, c] == 1:
                num_in_view += 1
                break
            if array[i, c] == 0:
                break
    
    # Top left diagonal
    if i > 0 and j > 0:
        rows = list(range(i-1, -1, -1))
        cols = list(range(j-1, -1, -1))        
        for r, c in zip(rows, cols):
            if array[r, c] == 1:
                num_in_view += 1
                break
            if array[r, c] == 0:
                break
    
    return num_in_view

In [11]:
def run_sims_2(state, num_calls=1):

    new_state = copy.deepcopy(state)     
    new_state = change_state(state, new_state, func=count_in_view, tol=5)
    
    if np.array_equal(new_state, state, equal_nan=True):
        return np.nansum(new_state), num_calls
    
    else:
#         print('round ', num_calls)
#         print(np.nansum(new_state))
#         print('state\n', state, '\nnew_state\n', new_state,'\n\n')
    
        return run_sims_2(new_state, num_calls+1)

In [12]:
run_sims_2(initial_state) # Should return 26

(26.0, 7)

In [17]:
%time run_sims_2(get_initial_state('seat_layout.txt'))

Wall time: 24.8 s


(1897.0, 84)